In [ ]:
#1
SELECT Произведение.Название
FROM  Произведение 
WHERE Произведение.id IN (SELECT Издание.id_произведения
                          FROM Издание 
                          GROUP BY Издание.id_произведения
                          HAVING COUNT(Издание.id_произведения) >=5 )

In [ ]:
#2
SELECT Экземпляр.инвентаризационный номер
FROM Экземпляр LEFT JOIN Издание ON Экземпляр.id_издания = Издание.id
WHERE Издание.id IS NULL

In [ ]:
#3
#Посчет взятых пользователем книг от последней к первой
WITH rn_date AS(
SELECT id_user,
       ROW_NUMBER() OVER(PARTITION BY id_user ORDER BY дата_взяли DESC AS rn,
       id_экземпляра
FROM Лог_операций))

#Посчет кол-во взятий каждого экземпляра
count_take AS(
SELECT id_экземпляра,COUNT(id_экземпляра) AS "кол-во взятий" 
FROM Лог_операций
GROUP BY id_экземпляра)

SELECT id_user,id_экземпляра,"кол-во взятий"
FROM rn_date INNER JOIN count_take USING(id_экземпляра)
WHERE rn < 4

In [ ]:
#4
# Пусть установленный библиотекой срок пользования книгой  = 20 дней, тогда критерии для рейтинга 
# неблагонадежных пользователей могут быть следующие:
# 1 Количество невозвратов > 3
# 2 Количество возвратов не в установленный срок  > 5


#кол-во невозвратов
WITH count_noreturn AS(
SELECT id_user,COUNT(id_user) AS "кол-во невозвратов"
FROM Лог_операций
WHERE DATEDIFF(days,DATEADD(days,20,дата_взяли),GETDATE()) > 20 AND дата_вернули IS NULL
GROUP BY id_user
HAVING COUNT(id_user) > 3
)
#кол-во просрочек
count_overdue AS (
SELECT id_user,COUNT(id_user) AS "кол-во просрочек"
FROM Лог_операций
WHERE DATEDIFF(days,дата_вернули,дата_взяли) > 20
GROUP BY id_user
HAVING COUNT(id_user) > 5
)

SELECT 
       COALESCE(count_overdue.id_user,count_noreturn.id_user) AS id_user,
       IFNULL("кол-во невозвратов",0) AS "кол-во невозвратов",
       IFNULL("кол-во просрочек",0) AS "кол-во просрочек"
FROM count_noreturn FULL OUTER JOIN count_overdue USING(id_user)
ORDER BY "кол-во невозвратов" + "кол-во просрочек" DESC
LIMIT 10